# Building a Postgres DB for Crime Reports

>The project demonstrates the process of creating an optimal database named crimes_db to store data on crime reports primarily available in the CSV format.

## 1. Setup

In [4]:
!pip3 install psycopg2-binary

In [43]:
import psycopg2
import csv
import pandas as pd

>Connecting to the default postgres db with default postgres user

In [9]:
conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()

>Creating a new database crime_db

In [10]:
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db;")
conn.close()

>Connecting to the new db and creating the schema

In [12]:
conn = psycopg2.connect("dbname=crime_db user=postgres")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

>Reading the data

In [58]:
with open("data/boston.csv", "r") as f:
    reader = csv.reader(f)
    col_headers = next(f)
    first_row = next(f)
print("Headers: ",col_headers)
print("First Row: ",first_row)
# converting col headers to a list
col_headers = col_headers.split(",")

Headers:  incident_number,offense_code,description,date,day_of_the_week,lat,long

First Row:  1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.35779134,-71.13937053



## 2. ETL Process

### 2.1 Finding Optimal Data Types for PostgreSQL

>The function get_unique() computes unique values for a given column. The aim of the method is to help determine the maximum length of existing text values and check whether an enumerated data type can be used.

In [78]:
def get_unique(file, column_index):
    f = pd.read_csv(file)
    col = f.iloc[:,column_index]
    output = set(col)
    return output

>Computing the number of unique values for each column

In [79]:
for i, col in enumerate(col_headers):
    data = get_unique("data/boston.csv", i)
    length = len(data)
    print(i, " ", col, " # of unique values: ",length)

0   incident_number  # of unique values:  298329
1   offense_code  # of unique values:  219
2   description  # of unique values:  239
3   date  # of unique values:  1177
4   day_of_the_week  # of unique values:  7
5   lat  # of unique values:  18177
6   long
  # of unique values:  18177


>Computing the value with maximum length for each column

In [77]:
out = dict()
for i, col in enumerate(col_headers):
    max_length = 0
    data = get_unique("data/boston.csv", i)
    for d in data:
        if isinstance(d, str):
            l = len(d)
            if max_length < l:
                max_length = l
            out.update({col:max_length})
out      

{'description': 58, 'date': 10, 'day_of_the_week': 9}

> **Outcomes**
>- The longest value in the **Day of the week** column contains **9** characters, however, the column has only **7 unique** values, therefore, it can be converted to an **enumerated** column.
>- The longest value in the **Description column** contains **58** characters. An optimal type could be **VARCHAR(100)**.
>- The **date** column can be converted to **date** type.
>- The rest of columns shall be numeric.

### 2.2 Building Tables in PostgreSQL

In [84]:
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long\n']

In [90]:
first_row

'1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.35779134,-71.13937053\n'

>Creating ENUM type for days of the week

In [83]:
cur.execute("""
CREATE TYPE enum_weekday
AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
""")
conn.commit()

>Building the table boston_crimes with appropriate types

In [86]:
cur.execute("""
CREATE TABLE crimes.boston_crimes (
    incident_number integer PRIMARY KEY,
    offense_code integer,
    description varchar(100),
    date date,
    day_of_the_week enum_weekday,
    lat double precision,
    long double precision
);
""")
conn.commit()

In [89]:
cur.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'boston_crimes'")
check_results = cur.fetchall()
check_results

[('incident_number',),
 ('offense_code',),
 ('description',),
 ('date',),
 ('day_of_the_week',),
 ('lat',),
 ('long',)]

### 2.3 Loading the Data into DB

In [91]:
with open("data/boston.csv", "r") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
conn.commit()

In [92]:
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 5;")
check_results = cur.fetchall()
check_results

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  42.35779134,
  -71.13937053),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  42.30682138,
  -71.06030035),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  42.34658879,
  -71.07242943),
 (4,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  42.33418175,
  -71.07866441),
 (5,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  42.27536542,
  -71.09036101)]

## 3. User & Groups Administration

### 3.1 Groups

>Following the least privilege principle the first step is to revoke all permissions from the **public group**.

In [105]:
cur.execute("""
    REVOKE ALL ON SCHEMA public FROM public; 
""")
cur.execute("""
    REVOKE ALL ON DATABASE crime_db FROM public; 
""")
conn.commit()

>Creating new User Groups and assigning corresponding roles

In [106]:
cur.execute("""
    CREATE GROUP readonly NOLOGIN;
""")
cur.execute("""
    CREATE GROUP readwrite NOLOGIN;
""")
conn.commit()

>Granting access to the database and schema usage rights to both groups

In [107]:
cur.execute("""
    GRANT CONNECT ON DATABASE crime_db TO readonly;
""")
cur.execute("""
    GRANT CONNECT ON DATABASE crime_db TO readwrite;
""")
cur.execute("""
    GRANT USAGE ON SCHEMA crimes TO readonly;
""")
cur.execute("""
    GRANT USAGE ON SCHEMA crimes TO readwrite;
""")
conn.commit()

>Granting specific group privileges

In [108]:
cur.execute("""
    GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;
""")
cur.execute("""
    GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;
""")
conn.commit()

### 3.2 Users

>Creating users and assigning them to groups

In [109]:
cur.execute("""
    CREATE USER data_analyst WITH PASSWORD 'secret1';
""")
cur.execute("""
    GRANT readonly TO data_analyst;
""")
conn.commit()

In [110]:
cur.execute("""
    CREATE USER data_scientist WITH PASSWORD 'secret2';
""")
cur.execute("""
    GRANT readwrite TO data_analyst;
""")
conn.commit()

### 3.3 Tests

In [111]:
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readonly';
""")
readonly_results = cur.fetchall()
readonly_results

[('readonly', 'SELECT')]

In [112]:
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';
""")
readwrite_results = cur.fetchall()
readwrite_results

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]